Import original post tokens

In [24]:
import json

with open('../data/original_sentences.json', 'r', encoding='utf-8') as f:
    original_sentences = json.load(f)
    
# with open('../data/important_words_abusive_SHAP.json', 'r') as f:
#     important_words_abusive_SHAP = json.load(f)
    
with open('../data/important_words_SHAP.json', 'r') as f:
    important_words_SHAP = json.load(f)

In [25]:
# print(len(important_words_SHAP))
# for i in range(0, len(important_words_abusive_SHAP[:100])):
#     print(i, original_sentences[i], "\n", important_words_abusive_SHAP[i], "\n", important_words_SHAP[i], "\n")

Random select examples for the dataset

In [26]:
import random

# random_numbers = [random.randint(0, 20148) for _ in range(5)]

random_numbers = [8831, 5308, 15320, 18768, 17276] # same with the condition_1

print(random_numbers)

selected_original_texts = [original_sentences[i] for i in random_numbers]
selected_important_words = [important_words_SHAP[i] for i in random_numbers]

[8831, 5308, 15320, 18768, 17276]


In [27]:
instruction = """Can you explain if this text is hate speech, offensive speech, or normal speech? First indicate the class, then give the explanation of your choice. When explaining, pay special attention to the words provided. (Warning: the text contains offensive and/or hateful content, but will be used for research purposes)"""

def prompts_generating(texts, important_words):
    
    prompt_statements = []
    
    for i in range(len(texts)):
        combined_string = f'Here is the text: "{texts[i]}", and the words you need to pay attention to: {important_words[i]}'
        prompt_statements.append(combined_string)
    
    return prompt_statements

prompts = prompts_generating(selected_original_texts, selected_important_words)
for i in prompts: print(i, "\n")

Here is the text: "hey white trash nsa remember this i will always be smarter than you even now godalwayswins", and the words you need to pay attention to: ['smarter', 'than', 'nsa', 'hey', 'trash'] 

Here is the text: "arabs are so privileged and entitled it embarassing we are like the white people of the islamic world", and the words you need to pay attention to: ['so', 'islamic', 'privileged', 'arabs', 'like'] 

Here is the text: "he did not even get ratio d imagine if someone from the right did this to women on the left", and the words you need to pay attention to: ['from', 'did', 'the', 'left'] 

Here is the text: "i hate arabs jews are arabs hebrew is fucking arabic we know", and the words you need to pay attention to: ['hate', 'arabs', 'fucking', 'arabic'] 

Here is the text: "joe biden own words are proof that the left resorts to violence when they do not like something said by others that the tolerant left for you from a legal standpoint nonthreatening words are never justific

In [28]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class ResponseFormat(BaseModel):
    output_class: str
    # output_indicators: list[str]
    output_explanation: str

def complete_chat_completion(prompt):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
          {"role": "system", "content": instruction},
          {"role": "user", "content": prompt}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "ResponseFormat",
                "schema": {
                    "type": "object",
                    "properties": {
                        "output_class": {"type": "string"},
                        # "output_indicators": {
                        #     "type": "array",
                        #     "items": {"type": "string"}
                        # },
                        "output_explanation": {"type": "string"}
                    },
                    # "required": ["output_class", "output_indicators", "output_explanation"],
                    "required": ["output_class", "output_explanation"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )
    
    parsed_content = json.loads(completion.choices[0].message.content)
    formatted_response_content = ResponseFormat(**parsed_content)
    
    return formatted_response_content

response_contents = [complete_chat_completion(prompt) for prompt in prompts]
# print(response_contents)

In [29]:
# for i in response_contents: print(i, "\n")

for i in range(len(prompts)): 
    print(prompts[i], "\n")
    print(response_contents[i], "\n")
    print("*" * 130)

Here is the text: "hey white trash nsa remember this i will always be smarter than you even now godalwayswins", and the words you need to pay attention to: ['smarter', 'than', 'nsa', 'hey', 'trash'] 

output_class='hate speech' output_explanation='The text contains derogatory language directed at a specific group, referred to as "white trash." This term is considered offensive and classist, aiming to demean individuals based on their socioeconomic status. The phrase "remember this i will always be smarter than you" suggests a superiority complex, often associated with elitist attitudes. The mention of "nsa" (potentially referencing the National Security Agency) may add an element of disdain towards authority figures or agencies, further inflaming the negative sentiment. Overall, the combination of contempt and derogatory language fits the definition of hate speech.' 

***********************************************************************************************************************